<a href="https://colab.research.google.com/github/berkebeyazbenli/BertProjectAi/blob/main/PdfAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PyMuPDF transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 35.7 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [19]:
import fitz

def read_pdf(file_path):
    document = fitz.open(file_path)
    text = ""
    for page_num in range(len(document)):
        page = document.load_page(page_num)
        text += page.get_text()
    return text

pdf_path = '/content/drive/My Drive/Alarko Holding.pdf'
pdf_text = read_pdf(pdf_path)
print(pdf_text[:1000])


Alarko Holding, İshak Alaton ve Üzeyir Garih tarafından 1972 yılında 
kurulan, altyapı inşaatı, enerji, finans, gayrimenkul, hafif 
sanayi ve turizm sektörlerinde faaliyet yürüten İstanbul merkezli holdingdir. 
Alarko ismi; AL alım ve satım AR araştırma ve geliştirme, KO komple tesisler kurma 
anlamına gelmektedir.[3] 
1954 yılında İshak Alaton ve Üzeyir Garih tarafından İstanbul'da bulunan 'Bankalar 
Caddesi Vefai Hanı’nda kolektif bir şirket olarak kurulmuştur. İlk yıllarda ana iş kolunu 
ısıtma, klima, soğutma işleri oluşturmuştur. 1973 yılında holding haline gelmiş, 1974 
yılında halka açılmıştır. 1997 yılında holding bünyesinde kurulan Alarko Gayrimenkul 
Yatırım Ortaklığı İMKB’de işlem görmeye başlayan ilk GYO şirketidir. 1998 yılında 
holdingin sanayi grubu dünyanın en büyük iklimlendirme firmalarından Carrier ile eşit 
oranda ortak olmuştur. 
2014 yılı itibarıyla taahhüt, enerji, sanayi, turizm, su ürünleri ve gayrimenkul 
alanlarında faaliyet göstermektedir. Ayrıca 1986 yılınd

In [24]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch

tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
model = AutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

def answer_question(question, context):
    inputs = tokenizer.encode_plus(question, context, add_special_tokens=True, return_tensors="pt", truncation=True, max_length=512)
    input_ids = inputs["input_ids"]
    attention_mask = inputs["attention_mask"]

    outputs = model(input_ids, attention_mask=attention_mask)
    answer_start_scores = outputs.start_logits
    answer_end_scores = outputs.end_logits

    answer_start = torch.argmax(answer_start_scores)
    answer_end = torch.argmax(answer_end_scores) + 1

    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[0][answer_start:answer_end]))
    return answer

def split_text(text, max_length):
    sentences = text.split('. ')
    current_chunk = []
    current_length = 0
    for sentence in sentences:
        sentence_length = len(tokenizer.encode(sentence, add_special_tokens=False))
        if current_length + sentence_length + 1 <= max_length:
            current_chunk.append(sentence)
            current_length += sentence_length + 1
        else:
            yield '. '.join(current_chunk) + '.'
            current_chunk = [sentence]
            current_length = sentence_length + 1
    if current_chunk:
        yield '. '.join(current_chunk) + '.'

question = "Alarko ne zaman kurulmuştur?"
max_chunk_length = 450  # Maksimum token sayısına uygun şekilde bir parça uzunluğu belirleyin
context_chunks = list(split_text(pdf_text, max_chunk_length))

for i, chunk in enumerate(context_chunks):
    print(f"Chunk {i+1}/{len(context_chunks)}:")
    print(answer_question(question, chunk))
    print()


Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Chunk 1/2:
[CLS]

Chunk 2/2:
alarko egitim - kultur vakfı

